In [1]:
import requests
import json
import pymongo
from key import key
from pprint import pprint
url = "https://api-basketball.p.rapidapi.com/"
headers = {
    "X-RapidAPI-Host": "api-basketball.p.rapidapi.com",
    "X-RapidAPI-Key": key
}

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


In [3]:
db = client.nba_db

In [4]:
seasons = ['2019-2020', '2020-2021', '2021-2022']
collection = db.standings
collection.delete_many({})
for season in seasons:
    params = f'standings?league=12&season={season}'
    response = requests.get(url + params, headers=headers).json()

    weststandings = {}
    eaststandings = {}

    for team in response["response"][0]:

        if team["group"]["name"] == "Western Conference":
            weststandings[team["team"]["name"]] = {"wins": team["games"]["win"]["total"],
                                  "losses": team["games"]["lose"]["total"], "logo": team["team"]["logo"]}
        if team["group"]["name"] == "Eastern Conference":
            eaststandings[team["team"]["name"]] = {"wins": team["games"]["win"]["total"],
                                  "losses": team["games"]["lose"]["total"], "logo" : team["team"]["logo"]}
    collection.insert_one({"season" : season, "data" : {"weststandings" : weststandings, "eaststandings": eaststandings}})



In [5]:
idmap = {}
for team in response["response"][0]:
    idmap[team["team"]["id"]] = team["team"]["name"]

In [10]:
collection = db.teamlist
teams = []
for team in range(132, 162):
    teams.append(idmap[team])

print(teams)
collection.insert_one({"teamlist" : teams})

['Atlanta Hawks', 'Boston Celtics', 'Brooklyn Nets', 'Charlotte Hornets', 'Chicago Bulls', 'Cleveland Cavaliers', 'Dallas Mavericks', 'Denver Nuggets', 'Detroit Pistons', 'Golden State Warriors', 'Houston Rockets', 'Indiana Pacers', 'Los Angeles Clippers', 'Los Angeles Lakers', 'Memphis Grizzlies', 'Miami Heat', 'Milwaukee Bucks', 'Minnesota Timberwolves', 'New Orleans Pelicans', 'New York Knicks', 'Oklahoma City Thunder', 'Orlando Magic', 'Philadelphia 76ers', 'Phoenix Suns', 'Portland Trail Blazers', 'Sacramento Kings', 'San Antonio Spurs', 'Toronto Raptors', 'Utah Jazz', 'Washington Wizards']


In [26]:
collection = db.h2h
collection.delete_many({})
for team1 in range(132, 162):
    for team2 in range(team1 + 1, 162):
        games = []
        params = f"/games?season=2020-2021&h2h={team1}-{team2}"
        response = requests.get(url + params, headers=headers).json()["response"]
        for game in response:
            simplegame = {"date" : game["date"][:game["date"].index('T')], 
                          "home" : {"team": game["teams"]["home"]["name"], "score": game["scores"]["home"]["total"]},
                          "away" : {"team": game["teams"]["away"]["name"], "score": game["scores"]["away"]["total"]}}
            games.append(simplegame)
        collection.insert_one({'teams': [idmap[team1], idmap[team2]], "data": games})
        

In [4]:
with open('player_stats.json') as file:
    file_data = json.load(file)

In [7]:
collection = db.player_data
collection.insert_many(file_data)

In [23]:
games = []
params = "/games?season=2020-2021&h2h=132-134"
response = requests.get(url + params, headers=headers).json()["response"]
for game in response:
    simplegame = {"date" : game["date"][:game["date"].index('T')], "home" : {"team": game["teams"]["home"]["name"], "score": game["scores"]["home"]["total"]},
                  "away" : {"team": game["teams"]["away"]["name"], "score": game["scores"]["away"]["total"]}}
    games.append(simplegame)
h2h = {'teams': [idmap[132], idmap[134]], "data": games}
pprint(h2h)

{'data': [{'away': {'score': 141, 'team': 'Atlanta Hawks'},
           'date': '2020-12-31',
           'home': {'score': 145, 'team': 'Brooklyn Nets'}},
          {'away': {'score': 114, 'team': 'Atlanta Hawks'},
           'date': '2021-01-02',
           'home': {'score': 96, 'team': 'Brooklyn Nets'}},
          {'away': {'score': 132, 'team': 'Brooklyn Nets'},
           'date': '2021-01-28',
           'home': {'score': 128, 'team': 'Atlanta Hawks'}}],
 'teams': ['Atlanta Hawks', 'Brooklyn Nets']}


In [12]:
pprint(response)

[{'country': {'code': 'US',
              'flag': 'https://media.api-sports.io/flags/us.svg',
              'id': 5,
              'name': 'USA'},
  'date': '2015-11-05T01:00:00+00:00',
  'id': 26090,
  'league': {'id': 12,
             'logo': 'https://media.api-sports.io/basketball/leagues/12.png',
             'name': 'NBA',
             'season': '2015-2016',
             'type': 'League'},
  'scores': {'away': {'over_time': None,
                      'quarter_1': 17,
                      'quarter_2': 26,
                      'quarter_3': 27,
                      'quarter_4': 17,
                      'total': 87},
             'home': {'over_time': None,
                      'quarter_1': 23,
                      'quarter_2': 18,
                      'quarter_3': 31,
                      'quarter_4': 29,
                      'total': 101}},
  'stage': None,
  'status': {'long': 'Game Finished', 'short': 'FT', 'timer': None},
  'teams': {'away': {'id': 134,
                

In [ ]:
f"/games?season=2020-2021&h2h={team1}-{team2}"